<a href="https://colab.research.google.com/github/Jay-cell427/Jay-cell427/blob/main/Biogas_Prototype_3D_Visualization_Script_(Blender).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import bpy

# --- Scene Setup ---
def clean_scene():
    """Cleans up the default scene objects."""
    bpy.ops.object.select_all(action='DESELECT')
    if 'Cube' in bpy.data.objects:
        bpy.data.objects['Cube'].select_set(True)
    if 'Light' in bpy.data.objects:
        bpy.data.objects['Light'].select_set(True)
    if 'Camera' in bpy.data.objects:
        bpy.data.objects['Camera'].select_set(True)
    bpy.ops.object.delete()

    # Create a new light for better visibility
    bpy.ops.object.light_add(type='SUN', location=(5, -5, 10))
    bpy.data.objects['Sun'].data.energy = 2.0

    # Set up a camera
    bpy.ops.object.camera_add(location=(15, -15, 10))
    bpy.context.scene.camera = bpy.context.object
    bpy.context.object.rotation_euler = (0.9, 0, 0.7) # Adjust camera angle

def create_material(name, color):
    """Creates a simple diffuse material with a given color."""
    mat = bpy.data.materials.new(name=name)
    mat.diffuse_color = (*color, 1.0) # RGB + Alpha
    return mat

# --- Component Creation Functions ---
def create_cylinder(name, radius, depth, location, rotation=(0,0,0), material=None):
    """Creates a cylinder and applies properties."""
    bpy.ops.mesh.primitive_cylinder_add(
        radius=radius, depth=depth,
        location=location,
        rotation=rotation
    )
    obj = bpy.context.object
    obj.name = name
    if material:
        if obj.data.materials:
            obj.data.materials[0] = material
        else:
            obj.data.materials.append(material)
    return obj

def create_cube(name, size, location, material=None):
    """Creates a cube and applies properties."""
    bpy.ops.mesh.primitive_cube_add(
        size=size,
        location=location
    )
    obj = bpy.context.object
    obj.name = name
    if material:
        if obj.data.materials:
            obj.data.materials[0] = material
        else:
            obj.data.materials.append(material)
    return obj

def create_pipe_segment(name, start_loc, end_loc, radius, material):
    """Creates a pipe segment between two points."""
    # Calculate midpoint and direction
    mid_x = (start_loc[0] + end_loc[0]) / 2
    mid_y = (start_loc[1] + end_loc[1]) / 2
    mid_z = (start_loc[2] + end_loc[2]) / 2

    vec = (end_loc[0] - start_loc[0], end_loc[1] - start_loc[1], end_loc[2] - start_loc[2])
    length = (vec[0]**2 + vec[1]**2 + vec[2]**2)**0.5

    if length == 0:
        print(f"Warning: Zero length pipe segment for {name}. Skipping.")
        return None

    # Calculate rotation to align cylinder with vector
    from math import acos, atan2
    from mathutils import Vector

    # Default cylinder is along Z-axis (0,0,1)
    default_axis = Vector((0, 0, 1))
    target_axis = Vector(vec).normalized()

    # Calculate rotation axis and angle
    rotation_axis = default_axis.cross(target_axis)
    rotation_angle = acos(default_axis.dot(target_axis))

    # Handle colinear case (no cross product)
    if rotation_axis.length < 1e-6: # Check if axis is almost zero
        if default_axis.dot(target_axis) < 0: # If opposite direction
            rotation_axis = Vector((1, 0, 0)) # Use any perpendicular axis
            rotation_angle = 3.14159 # 180 degrees
        else: # Same direction, no rotation needed
            rotation_axis = Vector((0, 0, 1)) # Dummy axis
            rotation_angle = 0

    # Create cylinder
    bpy.ops.mesh.primitive_cylinder_add(
        radius=radius, depth=length,
        location=(mid_x, mid_y, mid_z)
    )
    obj = bpy.context.object
    obj.name = name

    # Apply rotation
    # Convert axis-angle to quaternion, then to euler
    from mathutils import Quaternion
    quat = Quaternion(rotation_axis, rotation_angle)
    obj.rotation_mode = 'QUATERNION'
    obj.rotation_quaternion = quat

    if material:
        if obj.data.materials:
            obj.data.materials[0] = material
        else:
            obj.data.materials.append(material)
    return obj

# --- Main Prototype Assembly ---
def create_biogas_prototype():
    clean_scene()

    # --- Materials ---
    tank_mat = create_material("TankMaterial", (0.3, 0.5, 0.7)) # Blue-grey
    pipe_mat = create_material("PipeMaterial", (0.5, 0.5, 0.5)) # Grey
    sensor_mat = create_material("SensorMaterial", (0.9, 0.2, 0.2)) # Red
    ground_mat = create_material("GroundMaterial", (0.2, 0.7, 0.2)) # Green
    biochar_mat = create_material("BiocharMaterial", (0.2, 0.2, 0.2)) # Dark
    alkanolamine_mat = create_material("AlkanolamineTankMaterial", (0.7, 0.5, 0.3)) # Orange-brown
    gas_storage_mat = create_material("GasStorageMaterial", (0.7, 0.7, 0.3)) # Yellowish

    # --- Ground Plane ---
    ground = create_cube("GroundPlane", 30, (0, 0, -0.1), material=ground_mat)
    ground.scale.z = 0.01 # Make it flat

    # --- Main Biodigester Tank ---
    # Assume it's partially buried, so its base is slightly below ground (z=0)
    main_digester_loc = (0, 0, 2.5)
    main_digester = create_cylinder("MainBiodigester", radius=3, depth=5, location=main_digester_loc, material=tank_mat)

    # --- Underground Reservoir Tanks ---
    # These will be mostly below the ground plane (z < 0)
    res_tank_radius = 1.5
    res_tank_depth = 4
    underground_offset_z = -res_tank_depth / 2 - 0.5 # Offset so top is slightly below ground

    res_tank1_loc = (-6, 0, underground_offset_z)
    res_tank2_loc = (6, 0, underground_offset_z)
    res_tank3_loc = (0, -6, underground_offset_z)
    res_tank4_loc = (0, 6, underground_offset_z)

    res_tank1 = create_cylinder("UndergroundReservoir1", res_tank_radius, res_tank_depth, res_tank1_loc, material=tank_mat)
    res_tank2 = create_cylinder("UndergroundReservoir2", res_tank_radius, res_tank_depth, res_tank2_loc, material=tank_mat)
    res_tank3 = create_cylinder("UndergroundReservoir3", res_tank_radius, res_tank_depth, res_tank3_loc, material=tank_mat)
    res_tank4 = create_cylinder("UndergroundReservoir4", res_tank_radius, res_tank_depth, res_tank4_loc, material=tank_mat)

    # --- Aqua-biomass Inlet ---
    inlet_loc = (main_digester_loc[0] + 3.5, main_digester_loc[1] + 1, main_digester_loc[2] + 2)
    inlet_pipe = create_cylinder("AquaBiomassInletPipe", radius=0.4, depth=2, location=inlet_loc, rotation=(1.57, 0, 0), material=pipe_mat) # Rotated to be angled
    inlet_funnel = create_cylinder("AquaBiomassInletFunnel", radius=0.8, depth=0.5, location=(inlet_loc[0], inlet_loc[1], inlet_loc[2] + 1.2), material=pipe_mat)

    # --- Alkanolamine Side Tanks and Feedback Mechanism ---
    alkanolamine_tank_radius = 1
    alkanolamine_tank_depth = 3
    alkanolamine_tank1_loc = (main_digester_loc[0] - 5, main_digester_loc[1] + 4, main_digester_loc[2] + 1)
    alkanolamine_tank2_loc = (main_digester_loc[0] - 5, main_digester_loc[1] + 6, main_digester_loc[2] + 1) # For selling
    alkanolamine_feedback_tank_loc = (main_digester_loc[0] - 5, main_digester_loc[1] + 2, main_digester_loc[2] + 1)

    alk_tank1 = create_cylinder("AlkanolamineGenTank1", alkanolamine_tank_radius, alkanolamine_tank_depth, alkanolamine_tank1_loc, material=alkanolamine_mat)
    alk_tank2 = create_cylinder("AlkanolamineSalesTank", alkanolamine_tank_radius, alkanolamine_tank_depth, alkanolamine_tank2_loc, material=alkanolamine_mat)
    alk_feedback_tank = create_cylinder("AlkanolamineFeedbackTank", alkanolamine_tank_radius, alkanolamine_tank_depth, alkanolamine_feedback_tank_loc, material=alkanolamine_mat)

    # --- Gas Storage Tank ---
    gas_storage_loc = (main_digester_loc[0] + 6, main_digester_loc[1], main_digester_loc[2] + 1)
    gas_storage_tank = create_cylinder("GasStorageTank", radius=2.5, depth=4, location=gas_storage_loc, material=gas_storage_mat)

    # --- Biochar and Manure Side Tanks ---
    manure_tank_radius = 1.2
    manure_tank_depth = 2.5
    manure_tank1_loc = (main_digester_loc[0], main_digester_loc[1] - 5, main_digester_loc[2] - 0.5)
    manure_tank2_loc = (main_digester_loc[0] + 2, main_digester_loc[1] - 5, main_digester_loc[2] - 0.5)

    biochar_tank = create_cylinder("BiocharCollectionTank", manure_tank_radius, manure_tank_depth, manure_tank1_loc, material=biochar_mat)
    manure_tank = create_cylinder("ManureCollectionTank", manure_tank_radius, manure_tank_depth, manure_tank2_loc, material=biochar_mat) # Assuming same material for now

    # --- Piping System and Sensors ---
    pipe_radius = 0.15
    sensor_size = 0.3

    # Main Digester to Underground Reservoirs (drainage/slurry inlet)
    create_pipe_segment("Pipe_Res1_Digester", res_tank1_loc, main_digester_loc, pipe_radius, pipe_mat)
    create_pipe_segment("Pipe_Res2_Digester", res_tank2_loc, main_digester_loc, pipe_radius, pipe_mat)
    create_pipe_segment("Pipe_Res3_Digester", res_tank3_loc, main_digester_loc, pipe_radius, pipe_mat)
    create_pipe_segment("Pipe_Res4_Digester", res_tank4_loc, main_digester_loc, pipe_radius, pipe_mat)

    # Main Digester to Gas Storage Tank (biogas output)
    pipe_digester_gas_start = (main_digester_loc[0], main_digester_loc[1] - 0.5, main_digester_loc[2] + 2.5) # Top of digester
    pipe_digester_gas_end = (gas_storage_loc[0] - 2.5, gas_storage_loc[1], gas_storage_loc[2] + 1) # Towards gas storage
    create_pipe_segment("Pipe_Digester_GasStorage_1", pipe_digester_gas_start, pipe_digester_gas_end, pipe_radius, pipe_mat)
    create_pipe_segment("Pipe_Digester_GasStorage_2", pipe_digester_gas_end, gas_storage_loc, pipe_radius, pipe_mat) # To gas storage

    # Sensor on biogas output pipe
    sensor_loc_gas_out = ((pipe_digester_gas_start[0] + pipe_digester_gas_end[0]) / 2,
                          (pipe_digester_gas_start[1] + pipe_digester_gas_end[1]) / 2,
                          (pipe_digester_gas_start[2] + pipe_digester_gas_end[2]) / 2 + 0.3) # Slightly above pipe
    create_cube("Sensor_GasOutput", sensor_size, sensor_loc_gas_out, material=sensor_mat)

    # Main Digester to Alkanolamine Generation Tank
    pipe_digester_alk_start = (main_digester_loc[0] - 1, main_digester_loc[1] + 1, main_digester_loc[2] + 1)
    pipe_digester_alk_end = (alkanolamine_tank1_loc[0] + 0.5, alkanolamine_tank1_loc[1] - 0.5, alkanolamine_tank1_loc[2] + 1.5)
    create_pipe_segment("Pipe_Digester_Alkanolamine", pipe_digester_alk_start, pipe_digester_alk_end, pipe_radius, pipe_mat)

    # Alkanolamine Feedback Pipe (from feedback tank back to digester)
    pipe_alk_feedback_start = (alkanolamine_feedback_tank_loc[0] + 0.5, alkanolamine_feedback_tank_loc[1] + 0.5, alkanolamine_feedback_tank_loc[2] + 1.5)
    pipe_alk_feedback_end = (main_digester_loc[0] - 0.5, main_digester_loc[1] + 1.5, main_digester_loc[2])
    create_pipe_segment("Pipe_Alkanolamine_Feedback", pipe_alk_feedback_start, pipe_alk_feedback_end, pipe_radius, pipe_mat)

    # Alkanolamine Sales Pipe (from alkanolamine tank to sales tank)
    pipe_alk_sales_start = (alkanolamine_tank1_loc[0], alkanolamine_tank1_loc[1], alkanolamine_tank1_loc[2] + 1.5)
    pipe_alk_sales_end = (alkanolamine_tank2_loc[0], alkanolamine_tank2_loc[1], alkanolamine_tank2_loc[2] + 1.5)
    create_pipe_segment("Pipe_Alkanolamine_Sales", pipe_alk_sales_start, pipe_alk_sales_end, pipe_radius, pipe_mat)

    # Sensor on alkanolamine pipe
    sensor_loc_alk_pipe = ((pipe_digester_alk_start[0] + pipe_digester_alk_end[0]) / 2,
                            (pipe_digester_alk_start[1] + pipe_digester_alk_end[1]) / 2,
                            (pipe_digester_alk_start[2] + pipe_digester_alk_end[2]) / 2 + 0.3)
    create_cube("Sensor_Alkanolamine", sensor_size, sensor_loc_alk_pipe, material=sensor_mat)


    # Main Digester to Biochar/Manure Tanks (slurry/effluent output)
    pipe_digester_manure_start = (main_digester_loc[0] + 1.5, main_digester_loc[1], main_digester_loc[2] - 2.5) # Bottom of digester
    pipe_digester_manure_end_biochar = (biochar_tank.location[0], biochar_tank.location[1], biochar_tank.location[2] + 1.25)
    pipe_digester_manure_end_manure = (manure_tank.location[0], manure_tank.location[1], manure_tank.location[2] + 1.25)

    create_pipe_segment("Pipe_Digester_Biochar", pipe_digester_manure_start, pipe_digester_manure_end_biochar, pipe_radius, pipe_mat)
    create_pipe_segment("Pipe_Digester_Manure", pipe_digester_manure_start, pipe_digester_manure_end_manure, pipe_radius, pipe_mat)

    # Sensor on effluent pipe
    sensor_loc_manure_out = (pipe_digester_manure_start[0], pipe_digester_manure_start[1] - 0.5, pipe_digester_manure_start[2])
    create_cube("Sensor_Effluent", sensor_size, sensor_loc_manure_out, material=sensor_mat)

    print("Biogas Prototype 3D model created successfully in Blender!")

# --- Execute the script ---
if __name__ == "__main__":
    create_biogas_prototype()